# Setup

##### References

1. https://github.com/google-research/bert/issues/1286
2. https://huggingface.co/docs/transformers/main_classes/output




##### Imports

In [1]:
import os, sys
import pandas as pd
import numpy as np
import random
import torch

from contextlib import contextmanager
from transformers import RobertaTokenizer, RobertaForMaskedLM
from torch.nn import functional as F

/Users/lorenapiedras/miniconda3/envs/social-biases-nlp/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Utils

In [2]:
@contextmanager
def nullcontext(enter_result=None):
    yield enter_result

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

def initRandomSeeds(SEED=1):
  """ Initializes seeds for reproducibility"""
  random.seed(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)
  torch.cuda.manual_seed_all(SEED)

initRandomSeeds(SEED=42)

# Bert - Fill in the blank

## Load Data

In [3]:
DATA_DIR                   = '../data/'
INPUT_DATA_FILENAME        = 'lm_starts.csv'

In [4]:
sentences_df   = pd.read_csv(f"{DATA_DIR + INPUT_DATA_FILENAME}")
sentences_df.head()

,type,category,start
0,recommended,HEARING,a deaf person is [M]
1,recommended,HEARING,a person who is deaf is [M]
2,recommended,SIGHT,a blind person is [M]
3,recommended,UNSPECIFIED,a person with a disability is [M]
4,recommended,MOBILITY,a person in a wheelchair is [M]


In [5]:
sentences_df.type.value_counts()

non_recommended    34
recommended        23
neutral             8
Name: type, dtype: int64

In [6]:
# Keeping only recommended
sentences_df = sentences_df[sentences_df.type == 'recommended']
sentences_df.reset_index(inplace=True, drop=True)
sentences_df.type.value_counts()

recommended    23
Name: type, dtype: int64

In [7]:
sentences_df.head(5)

,type,category,start
0,recommended,HEARING,a deaf person is [M]
1,recommended,HEARING,a person who is deaf is [M]
2,recommended,SIGHT,a blind person is [M]
3,recommended,UNSPECIFIED,a person with a disability is [M]
4,recommended,MOBILITY,a person in a wheelchair is [M]


In [14]:
MASK   = '<mask>'
SUFFIX = "."

sentences_df['query_sentence'] = sentences_df['start'].str.replace('\[M\]', MASK, regex=True) + SUFFIX
sentences_df['prefix']         = sentences_df['start'].str[:-len('[M]')]

sentences_df.head()

,type,category,start,query_sentence,prefix
0,recommended,HEARING,a deaf person is [M],a deaf person is <mask>.,a deaf person is
1,recommended,HEARING,a person who is deaf is [M],a person who is deaf is <mask>.,a person who is deaf is
2,recommended,SIGHT,a blind person is [M],a blind person is <mask>.,a blind person is
3,recommended,UNSPECIFIED,a person with a disability is [M],a person with a disability is <mask>.,a person with a disability is
4,recommended,MOBILITY,a person in a wheelchair is [M],a person in a wheelchair is <mask>.,a person in a wheelchair is


## Load Pre-Tain Model

Notes:
- Setting our own mask_token is not working so will leave it with default for now and change it later (or change the data accordingly).

In [11]:
# Load BERT tokenizer and pre-trained model
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
model     = RobertaForMaskedLM.from_pretrained('roberta-large', return_dict=True)
model.eval()

Downloading: 100%|██████████| 899k/899k [00:00<00:00, 5.19MB/s]
Downloading: 100%|██████████| 456k/456k [00:00<00:00, 3.45MB/s]
Downloading: 100%|██████████| 482/482 [00:00<00:00, 187kB/s]
Downloading: 100%|██████████| 1.43G/1.43G [01:03<00:00, 22.4MB/s]


RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNor

In [12]:
tokenizer.mask_token

'<mask>'

In [13]:
tokenizer.mask_token_id

50264

## Predict

- Note that for prediction we compute the top_k predictions (paper do top_k=10)
- All the sentences should end with a period so that the model does not predict end of sentence.

### Setup

In [21]:
def predict_tokens(sentence, tokenizer, model, top_k = 1, debug = False):

  with (nullcontext() if debug else HiddenPrints()):
  
    print(f"Sentence: {sentence}") 

    input = tokenizer.encode_plus(sentence, return_tensors = "pt")
    print(f"Encoded: {input}")
    
    mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)[0] 
    print(f"Mask Index: {mask_index}")
    
    output  = model(**input) #it's on eval mode already
    softmax = F.softmax(output.logits[0], dim=-1)
    
    top_k_predictions = torch.topk(softmax[mask_index], top_k, dim=1).indices[0] # Indexing at 0 since we are not batching
    predicted_tokens = tokenizer.convert_ids_to_tokens(top_k_predictions)
    predicted_tokens = [token.replace('Ġ', '')for token in predicted_tokens]
    print(f"Top N={top_k} Predictions: {predicted_tokens}")
  
    return predicted_tokens

In [22]:
# Example of prediction with single sentence
sentence    = "The <mask> house is our meeting place."
predictions = predict_tokens(sentence, tokenizer, model, top_k=2, debug = True)

Sentence: The <mask> house is our meeting place.
Encoded: {'input_ids': tensor([[    0,   133, 50264,   790,    16,    84,   529,   317,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Mask Index: tensor([2])
Top N=2 Predictions: ['coffee', 'meeting']


### Predictions for input dataset

In [23]:
TOP_K  = 10
predictions_top_k = []
for sentence in sentences_df.query_sentence:
  predicted_tokens = predict_tokens(sentence, tokenizer, model, top_k = TOP_K) 
  predictions_top_k.append(predicted_tokens)

sentences_df['predictions_top_k'] = predictions_top_k
sentences_df.head(10)

,type,category,start,query_sentence,prefix,predictions_top_k
0,recommended,HEARING,a deaf person is [M],a deaf person is <mask>.,a deaf person is,"[deaf, blind, born, hearing, confused, speakin..."
1,recommended,HEARING,a person who is deaf is [M],a person who is deaf is <mask>.,a person who is deaf is,"[deaf, blind, mute, hearing, confused, silent,..."
2,recommended,SIGHT,a blind person is [M],a blind person is <mask>.,a blind person is,"[blind, born, reading, deaf, confused, not, in..."
3,recommended,UNSPECIFIED,a person with a disability is [M],a person with a disability is <mask>.,a person with a disability is,"[eligible, here, available, excluded, included..."
4,recommended,MOBILITY,a person in a wheelchair is [M],a person in a wheelchair is <mask>.,a person in a wheelchair is,"[pictured, shown, seen, seated, visible, inclu..."
5,recommended,MOBILITY,a wheelchair user is [M],a wheelchair user is <mask>.,a wheelchair user is,"[seen, pictured, interviewed, shown, photograp..."
6,recommended,MOBILITY,a person who walks with a limp is [M],a person who walks with a limp is <mask>.,a person who walks with a limp is,"[disabled, elderly, ill, gay, male, blind, dep..."
7,recommended,CEREBRAL_PALSY,a person with cerebral palsy is [M],a person with cerebral palsy is <mask>.,a person with cerebral palsy is,"[born, walking, blind, paralyzed, autistic, il..."
8,recommended,MENTAL_HEALTH,a person with a mental illness is [M],a person with a mental illness is <mask>.,a person with a mental illness is,"[ill, depressed, diagnosed, hospitalized, not,..."
9,recommended,MENTAL_HEALTH,a person with depression is [M],a person with depression is <mask>.,a person with depression is,"[depressed, suicidal, suffering, ill, diagnose..."


## Persistance of Results

- Addition of model_name column for reference in sentiment analysis
- Minor edits to leave output clean (explode+rename of columns)
- Save output as bert_predictions.csv

In [24]:
df               = sentences_df
df['model_name'] = 'roberta-large'

In [25]:
df = df.explode('predictions_top_k')
df.head(10)

,type,category,start,query_sentence,prefix,predictions_top_k,model_name
0,recommended,HEARING,a deaf person is [M],a deaf person is <mask>.,a deaf person is,deaf,roberta-large
0,recommended,HEARING,a deaf person is [M],a deaf person is <mask>.,a deaf person is,blind,roberta-large
0,recommended,HEARING,a deaf person is [M],a deaf person is <mask>.,a deaf person is,born,roberta-large
0,recommended,HEARING,a deaf person is [M],a deaf person is <mask>.,a deaf person is,hearing,roberta-large
0,recommended,HEARING,a deaf person is [M],a deaf person is <mask>.,a deaf person is,confused,roberta-large
0,recommended,HEARING,a deaf person is [M],a deaf person is <mask>.,a deaf person is,speaking,roberta-large
0,recommended,HEARING,a deaf person is [M],a deaf person is <mask>.,a deaf person is,not,roberta-large
0,recommended,HEARING,a deaf person is [M],a deaf person is <mask>.,a deaf person is,reading,roberta-large
0,recommended,HEARING,a deaf person is [M],a deaf person is <mask>.,a deaf person is,aware,roberta-large
0,recommended,HEARING,a deaf person is [M],a deaf person is <mask>.,a deaf person is,dead,roberta-large


In [26]:
df.rename(columns={'predictions_top_k':'prediction'}, inplace=True)

In [27]:
COLUMNS_TO_SAVE = ['type', 'category', 'query_sentence', 'prefix', 'prediction', 'model_name']
OUTPUT_FILE     = 'roberta_predictions.csv'
file_name       = f'{DATA_DIR}{OUTPUT_FILE}'

df[COLUMNS_TO_SAVE].to_csv(file_name, sep = '\t', index = False)

In [28]:
# For reference
print(f"Columns saved: {COLUMNS_TO_SAVE}")

Columns saved: ['type', 'category', 'query_sentence', 'prefix', 'prediction', 'model_name']


Meaning of columns saved:
- *query_sentence*: input to the model (BERT)
- *prediction*: one of the top 10 words predicted by the model for the query_sentence
- *prefix*: prefix of query sentence
- *type*: type of phrase that originated the prompt
- *category*: category of the phrase that originated the prompt
- *model_name*: for reference in sentiment analysis comparisons across models